In [41]:
import networkx as nx
import simplejson as json
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# parse the gml file and build the graph object
g = nx.read_gml('networks/data/karate.gml', label='id')
# create a dictionary in a node-link format that is suitable for JSON serialization
# d = json_graph.node_link_data(g)
# with open('networks/data/power_grid_westernus.json', 'w') as fp:
#     json.dump(d, fp)

In [ ]:
nx.adjacency_data(g)

In [ ]:
nx.draw(g, with_labels=True)

In [ ]:
vote_data = pd.read_csv('networks/data/house_votes_2022_curated.csv')

In [ ]:
vote_data["DATE"] = pd.to_datetime(vote_data["DATE"])

In [ ]:
vote_data["DATE"] = vote_data["DATE"].dt.date

In [ ]:
vote_data.sort_values(by=['DATE'], inplace=True)

In [ ]:
vote_data.reset_index(drop=True, inplace=True)

In [ ]:
vote_data["REPRESENTATIVE"] = vote_data["REPRESENTATIVE"].astype(str)

In [ ]:
vote_data["index"] = np.arange(len(vote_data))

In [ ]:
yea_nay_totals = vote_data.groupby(["BILL", "VOTE"]).count()

In [ ]:
yea_nay_totals

In [ ]:
reps_voted_yea_on_same_bill = vote_data[vote_data["VOTE"] == "YEA"].groupby(["BILL", "REPRESENTATIVE", "PARTY"]).count()

In [ ]:
reps_voted_nay_on_same_bill = vote_data[vote_data["VOTE"] == "NAY"].groupby(["BILL", "REPRESENTATIVE", "PARTY"]).count()

In [ ]:
reps_voted_same_on_bill = vote_data.groupby(["BILL", "REPRESENTATIVE", "PARTY", "VOTE", "DATE", "BILL_NAME", "COMMITTEE", "STATE"]).count()

In [ ]:
voted_same_df = reps_voted_same_on_bill.reset_index()

In [ ]:
reps_voted_same_on_bill.to_csv('networks/data/reps_voted_same_on_bill.csv')

In [ ]:
def create_node_link_json_from_vote_data(vote_data, reps_voted_yea_on_same_bill, reps_voted_nay_on_same_bill):
    nodes = []
    links = []
    for index, row in vote_data.iterrows():
        print(index)
        nodes.append({"id": row["REPRESENTATIVE"], "group": row["PARTY"], "vote": row["VOTE"], "bill": row["BILL"], "date": row["DATE"].strftime("%Y-%m-%d")})
    for index, row in reps_voted_yea_on_same_bill.iterrows():
        links.append({"source": index[0], "target": index[1], "value": row["index"]})
    for index, row in reps_voted_nay_on_same_bill.iterrows():
        links.append({"source": index[0], "target": index[1], "value": row["index"]})
    return {"nodes": nodes, "links": links}

In [ ]:
def nodes_links_same_votes(vote_data, reps_voted_same_on_bill):
    nodes = []
    links = []
    for index, row in vote_data.iterrows():
        nodes.append({"id": row["REPRESENTATIVE"], "group": row["PARTY"], "vote": row["VOTE"], "bill": row["BILL"], "date": row["DATE"].strftime("%Y-%m-%d")})
    for index, row in reps_voted_same_on_bill.iterrows():
        links.append({"source": index[0], "target": index[1], "value": row["index"]})
    return {"nodes": nodes, "links": links}

In [ ]:
net_data = nodes_links_same_votes(vote_data, reps_voted_same_on_bill)

In [ ]:
net_data

In [ ]:
with open('networks/data/house_votes_2022_curated.json', 'w') as fp:
    json.dump(net_data, fp)

In [ ]:
def convert_yea_nay_to_binary(vote_data):
    vote_data["VOTE"] = vote_data["VOTE"].apply(lambda x: 1 if x == "YEA" else 0)
    return vote_data

In [ ]:
vote_data = convert_yea_nay_to_binary(vote_data)

In [62]:
def create_same_votes_heatmap(vote_data):
    same_votes = vote_data.groupby(["BILL", "REPRESENTATIVE", "PARTY", "VOTE"]).count()
    same_votes = same_votes.reset_index()
    same_votes = same_votes.pivot(index=["REPRESENTATIVE", "PARTY"], columns="BILL", values="VOTE")
    same_votes = same_votes.fillna(0)
    return same_votes

In [ ]:
vote_data

In [63]:
s_v = create_same_votes_heatmap(vote_data)

In [70]:
s_v

,BILL,1319,1808,3076,3617,3684,3755,5376,5746,7780,7791,7900,8404
REPRESENTATIVE,PARTY,,,,,,,,,,,,
Adams,Democratic,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA
Aderholt,Republican,NAY,NAY,YEA,NAY,NAY,NAY,NAY,NAY,NAY,YEA,YEA,NAY
Aguilar,Democratic,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA
Allen,Republican,NAY,NAY,NAY,NAY,NAY,NAY,NAY,NAY,NAY,YEA,YEA,NAY
Allred,Democratic,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wittman,Republican,NAY,NAY,YEA,NAY,NAY,NAY,NAY,NAY,NAY,YEA,YEA,NAY
Womack,Republican,NAY,NAY,YEA,NAY,NAY,NAY,NAY,NAY,NAY,YEA,YEA,NAY
Yarmuth,Democratic,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,YEA,NAY,YEA


In [ ]:
with open('networks/data/house_votes_network.json', 'w') as fp:
    json.dump(net_data, fp)